In [29]:
import pandas as pd
import requests
import json

In [33]:
def get_summoner_url(encrypted_summoner_id):
    return f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/{encrypted_summoner_id}?api_key={API_KEY}"

In [48]:
def get_match_id_url(puuid):
    return f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={API_KEY}"

In [60]:
def get_match_url(match_id):
    return f"https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={API_KEY}"

In [32]:
API_KEY = ""

In [17]:
res = requests.get(
    f"https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/GOLD/III?page=1&api_key={API_KEY}"
)
leagues = json.loads(res.content)
df = pd.DataFrame(leagues)

In [45]:
puuids = []
for sum_id in df.summonerId:
    res = requests.get(get_summoner_url(sum_id))
    puuids.append(json.loads(res.content)["puuid"])


KeyboardInterrupt



In [70]:
match_ids = []
for puuid in puuids:
    match_ids.extend(json.loads(requests.get(get_match_id_url(puuid)).content))

KeyboardInterrupt: 

In [71]:
match_ids = list(set(match_ids))

858

In [ ]:
matches = []
for match_id in match_ids:
    matches.append(json.loads(requests.get(get_match_url(match_id)).content))

In [92]:
cols = [
    "assists",
    "baronKills",
    "bountyLevel",
    "champExperience",
    "championId",
    "championName",
    "damageDealtToBuildings",
    "damageDealtToObjectives",
    "damageDealtToTurrets",
    "damageSelfMitigated",
    "totalDamageDealt",
    "totalDamageDealtToChampions",
    "totalDamageTaken",
    "totalEnemyJungleMinionsKilled",
    "totalHeal",
    "totalHealsOnTeammates",
    "totalMinionsKilled",
    "totalTimeCCDealt",
    "totalTimeSpentDead",
    "trueDamageDealt",
    "trueDamageDealtToChampions",
    "trueDamageTaken",
    "turretKills",
    "turretsLost",
    "win",
]

In [102]:
d = {}
for match in matches:
    for participant in match["info"]["participants"]:
        for col in cols:
            if col in d:
                d[col].append(participant[col])
            else:
                d[col] = [participant[col]]

KeyError: 'info'

In [105]:
pd.DataFrame(d)

,assists,baronKills,bountyLevel,champExperience,championId,championName,damageDealtToBuildings,damageDealtToObjectives,damageDealtToTurrets,damageSelfMitigated,...,totalHealsOnTeammates,totalMinionsKilled,totalTimeCCDealt,totalTimeSpentDead,trueDamageDealt,trueDamageDealtToChampions,trueDamageTaken,turretKills,turretsLost,win
0,1,0,0,6201,33,Rammus,664,664,664,16215,...,0,76,72,64,1875,0,456,0,3,False
1,3,0,0,6342,104,Graves,0,5323,0,8696,...,0,27,338,73,33970,408,184,0,3,False
2,1,0,0,6317,54,Malphite,1172,1172,1172,8969,...,0,82,270,121,1903,287,44,0,3,False
3,6,0,0,5444,22,Ashe,0,0,0,3401,...,0,114,386,76,3520,0,409,0,3,False
4,4,0,0,5533,53,Blitzcrank,0,0,0,7790,...,0,35,176,38,5568,306,84,0,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,0,0,0,9714,164,Camille,0,0,0,8607,...,0,137,123,99,16091,1682,0,0,10,False
714,6,0,0,9477,33,Rammus,0,12744,0,37093,...,168,49,345,105,41165,384,1614,0,10,False
715,7,0,0,8787,7,Leblanc,3516,3516,3516,6893,...,0,114,45,230,3408,603,502,1,10,False
716,2,0,0,7721,222,Jinx,703,1633,703,6891,...,0,99,43,86,2720,0,507,0,10,False


In [109]:
pd.DataFrame(d).to_csv("matches.csv", index=False)